In [0]:
%sql
show databases

In [0]:
spark.sql("USE globalretail_silver")

spark.sql("""
CREATE TABLE IF NOT EXISTS silver_customers (
    customer_id STRING,
    name STRING,
    email STRING,
    country STRING,
    customer_type STRING,
    registration_date DATE,
    age INT,
    gender STRING,
    total_purchases INT,
    customer_segment STRING,
    days_since_registration INT,
    last_updated TIMESTAMP
)
""")

In [0]:
%sql
show tables

In [0]:
# Get the last processed timestamp from silver layer
last_processed_df = spark.sql("select max(last_updated) as last_processed from silver_customers")
last_processed_timestamp = last_processed_df.collect()[0]['last_processed']

if last_processed_timestamp is None:
    last_processed_timestamp = '1900-01-01'

In [0]:
# Create a temporary view of incremental bronze data
spark.sql(f"""
          CREATE OR REPLACE TEMPORARY VIEW bronze_incremental_customers AS
          SELECT * FROM globalretail_bronze.bronze_customer WHERE ingestion_timestamp > '{last_processed_timestamp}'
          """)


In [0]:
%sql
select * from bronze_incremental_customers limit 10

In [0]:
#Validate email addresses (null or not null)
#Valid age between 18 to 100
#Create customer_segment as total_purchases > 10000 THEN 'High Value' if > 5000 THEN 'Medium Value' ELSE 'Low Value'
#days since user is registered in the system
#Remove any junk records where total_purchase is negative number

spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW silver_incremental_customers AS
SELECT
    customer_id,
    name,
    email,
    country,
    customer_type,
    registration_date,
    age,
    gender,
    total_purchases,
CASE
    WHEN total_purchases > 10000 THEN 'High Value'
    WHEN total_purchases > 5000 THEN 'Medium Value'
    ELSE 'Low Value'
END AS customer_segment,
DATEDIFF(CURRENT_DATE(), registration_date) AS days_since_registration,
CURRENT_TIMESTAMP() AS last_updated
FROM bronze_incremental_customers
WHERE
    age BETWEEN 18 AND 120
    AND email IS NOT NULL
    AND total_purchases >= 0
    """)

In [0]:
display(spark.sql("SELECT * FROM silver_incremental_customers"))

In [0]:
spark.sql("""
MERGE INTO silver_customers target
USING silver_incremental_customers source
ON target.customer_id = source.customer_id
WHEN MATCHED THEN
    UPDATE SET *
WHEN NOT MATCHED THEN
    INSERT *
""")